In [1]:
%load_ext autoreload
%autoreload 2

from helpers import *

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import GridSearchCV

path_dataset = "data/data_train.csv"
data = load_data(path_dataset)
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

data.head() 

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


![alt text](ALS.png "Title")

In [ ]:
from ALS_implementation import ALS
param_grid = {"n_epochs": [5], "num_features": [15, 25], "lambda_user": [0.05, 0.15], "lambda_item": [0.1, 0.9]}
gs = GridSearchCV(ALS, param_grid, measures=['rmse', 'mae'], cv=3) 
gs.fit(dataset)

number of items in preprocess_trainset_data: 1000, number of users: 10000

start the ALS algorithm...
RMSE on training set: 1.0287715148994026.
RMSE on training set: 0.9619197624562075.
RMSE on training set: 0.9315929134278401.
RMSE on training set: 0.9140482532086189.
RMSE on training set: 0.9027842213775896.
number of items in preprocess_trainset_data: 1000, number of users: 10000

start the ALS algorithm...
RMSE on training set: 1.0283325427298464.
RMSE on training set: 0.9632938389984093.
RMSE on training set: 0.9339118242913584.
RMSE on training set: 0.9153675179310596.
RMSE on training set: 0.9034083259712488.
number of items in preprocess_trainset_data: 1000, number of users: 10000

start the ALS algorithm...
RMSE on training set: 1.0283861954494506.
RMSE on training set: 0.9630651341268193.
RMSE on training set: 0.9335321751286696.
RMSE on training set: 0.9150986527000856.
RMSE on training set: 0.903225017381483.
number of items in preprocess_trainset_data: 1000, number of user